# Project 5: Prompting With Large Language Models

In this project, we learn how to solve tasks by prompting existing LLM APIs. We will experiment with zero-shot and few-shot prompting and different ways methods for example selection for a semantic parsing task.

First we install and import the required dependencies. These include:
* `openai` as our API for querying LLMs (you are free to choose to use a different LLM API if you would like)


In [1]:
%%capture
%pip install openai

If you are using the OpenAI API, then go to then create an account and then copy your secret API key from `https://platform.openai.com/account/api-keys`. Set this as an environment variable or key management service so we can load it below. Make to keep private key secret. You may use a different LLM service if you choose to eg. Cohere (https://cohere.ai/).

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ...
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

If you have successfully authorized, then you should be able to see a list of available models by running the command below.

In [3]:
import openai
openai.api_key = OPENAI_API_KEY
openai.Model.list()

<OpenAIObject list at 0x79b714e17e90> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

We will now evaluate the LLM on a semantic parsing task. Geoquery is a dataset that contains information about the geography of the United States. For more information, please see: https://www.cs.utexas.edu/users/ml/nldata/geoquery.html. We will experiment with the compositional split introduced in (Keysers et al., 2020) https://openreview.net/forum?id=SygcCnNKwr. First, let's download the train and validation data. The goal for the LLM is to take in English queries about US geography about population, elevation, etc. and output a formal representation of the query.

In [4]:
!wget https://github.com/kl2806/geoquery/raw/main/data.zip -O data.zip
!unzip -o data.zip

--2023-04-27 04:58:25--  https://github.com/kl2806/geoquery/raw/main/data.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip [following]
--2023-04-27 04:58:25--  https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4493 (4.4K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   4.39K  --.-KB/s    in 0s      

2023-04-27 04:58:25 (37.7 MB/s) - ‘data.zip’ saved [4493/4493]

Archive:  data.zip
  inflating: train.tsv               
  inflating: dev.tsv                 


Let's take a look at the data, each instance should compose of an English utterance, and a formal representation of the utterance

In [5]:
!head -n 5 ./train.tsv

how tall is the highest point in m0	answer ( elevation_1 ( highest ( intersection ( place , loc_2 ( m0 ) ) ) ) )
what is the largest city in m0	answer ( largest ( intersection ( city , loc_2 ( m0 ) ) ) )
what states border states that the m0 runs through	answer ( intersection ( state , next_to_2 ( intersection ( state , traverse_1 ( m0 ) ) ) ) )
what is the maximum elevation of m0	answer ( highest ( intersection ( place , loc_2 ( m0 ) ) ) )
what is the population of m0	answer ( population_1 ( m0 ) )


In [6]:
import csv
from dataclasses import dataclass
from typing import List

@dataclass
class Example:
     query: str
     program: str

training_examples: List[Example] = []

with open('./train.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        training_examples.append(Example(query, program))

dev_examples: List[Example] = []
with open('./dev.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        dev_examples.append(Example(query, program))

print(f"Num training examples: {len(training_examples)}")
print(f"Num dev examples: {len(dev_examples)}")


Num training examples: 440
Num dev examples: 40


Now, let's define a function that uses the OpenAI API to output the semantic parse. As a first cut, let's just try to describe the task in English the model and return it in `get_static_prompt`. Then implement `parse_example`, which should call the LLM API, and return the semantic parse. `gpt-3.5-turbo`, the corresponding API call to ChatGPT should work well for this task.

In [7]:
from typing import Callable

def get_static_prompt(utterance: str) -> str:
    """Return a prompt that doesn't change between different examples"""
    """YOUR CODE HERE"""
    return """Output a formal representation of the query.
Query: """+ utterance + """
Representation: 
"""
    
def parse_example(model: str, utterance: str, prompt_method: Callable[[str], str], **kwargs: dict) -> str:
    """Return the semantic parse of the utterance"""
    prompt = prompt_method(utterance, **kwargs)
    """YOUR CODE HERE"""
    conversation = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model, messages=conversation)
    return response.choices[0].message.content

parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=get_static_prompt)

print(parse)


SELECT river_name 
FROM rivers 
WHERE city_code = 'm0';


With just an English description, the output probably does not look very similar to the target language that we want. Let's try to construct a prompt with some examples from our training set and see how it does. Implement the function below for uniformly sampling examples from the training set, and using that to as a few-shot prompt to the model.

In [8]:
import random
from typing import List

def random_sample_prompt(utterance: str, training_examples: List[Example], num_samples: int = 10) -> str:
    """Return a prompt for a given example"""
    """YOUR CODE HERE"""
    sampled_examples = random.sample(training_examples, num_samples)
    prompt = """Given these examples:
"""
    for example in sampled_examples:
        prompt += "Query: " + example.query + "\nRepresentation: " + example.program + "\n"
    prompt += """Output a formal representation of the following query.
Query: """+ utterance + """
Representation:
"""
    return prompt
    

prompt = random_sample_prompt(utterance="what river runs through m0",
                              training_examples=training_examples)
print("Uniform random sampling prompt: \n", prompt)


Uniform random sampling prompt: 
 Given these examples:
Query: what is the population of m0
Representation: answer ( population_1 ( m0 ) )
Query: what is the area of m0
Representation: answer ( area_1 ( m0 ) )
Query: how long is the m0 river
Representation: answer ( len ( intersection ( river , m0 ) ) )
Query: how many people live in m0
Representation: answer ( population_1 ( m0 ) )
Query: how many citizens in m0
Representation: answer ( population_1 ( m0 ) )
Query: how many rivers are there in m0
Representation: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Query: which is the longest river in m0
Representation: answer ( longest ( intersection ( river , loc_2 ( m0 ) ) ) )
Query: what is the capital of m0
Representation: answer ( intersection ( capital , loc_2 ( m0 ) ) )
Query: what is capital of m0
Representation: answer ( intersection ( capital , loc_2 ( m0 ) ) )
Query: which states border m0
Representation: answer ( intersection ( state , next_to_2 ( m0 ) ) )
Output a f

In [9]:
parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=random_sample_prompt,
                      training_examples=training_examples)
print(parse)

answer ( intersection ( river , loc_2 ( m0 ) ) )


Now, let's evaluate our uniform sampling prompt on the validation set. If you run into rate limit issues with the API, you may want to backoff or consult one of the solutions here https://platform.openai.com/docs/guides/rate-limits/error-mitigation.

In [10]:
import tqdm

def get_predictions(model: str,
                    evaluation_examples: List[Example],
                    prompt_creation_function: Callable[[str], str],
                    **kwargv: List[str]) -> List[str]:
    """Get a list of predictions from the evaluation examples"""
    predictions = []
    for example in tqdm.tqdm(evaluation_examples):
        predicted_program = parse_example(model, example.query, prompt_creation_function, **kwargv)
        predictions.append(predicted_program)
    return predictions

def evaluate(predictions: List[str], evaluation_examples: List[Example]) -> float:
    """Evaluate the accuracy of the predictions"""
    correct = 0
    for prediction, example in zip(predictions, evaluation_examples):
        if prediction == example.program:
            correct += 1
    return correct / len(evaluation_examples)

In [11]:
random_sample_predictions = get_predictions(model="gpt-3.5-turbo",
                                            evaluation_examples=dev_examples,
                                            prompt_creation_function=random_sample_prompt,
                                            training_examples=training_examples)


100%|██████████| 40/40 [01:49<00:00,  2.75s/it]


The model should get at least 15\% exact match with randomly sampling examples.

In [12]:

# Save the predictions as `random_predictions.txt`
with open('random_predictions.txt', 'w') as f:
    for prediction in random_sample_predictions:
        f.write(prediction)
        f.write('\n')

exact_match = evaluate(random_sample_predictions, dev_examples)
print(f"Exact match for uniform sampling prompt: {exact_match}")

Exact match for uniform sampling prompt: 0.375


Randomly sampling examples does not use consider the utterance when selecting examples. Next, we will try to pick examples for the prompt based on embedding similarity. First, let's install `sentence-transformers`, which we will use to get embeddings of the utterance.

In [13]:
%%capture
%pip install -U sentence-transformers

Now, let's construct embeddings using a small pretrained model for each example in our training data.

In [19]:

from sentence_transformers import SentenceTransformer, util
import torch

def get_corpus(model_name: str, examples: List[Example]) -> torch.Tensor:
    """Return a tensor of the corpus embeddings, the size of the returned tensor should be (num_examples, embedding_size)"""
    """YOUR CODE HERE"""
    model = SentenceTransformer(model_name)
    if isinstance(examples, str):
        examples_utt = examples
    else:
        examples_utt = [example.query for example in examples]
    embeddings = model.encode(examples_utt)
    return torch.tensor(embeddings)

corpus_embeddings = get_corpus('all-MiniLM-L6-v2', training_examples)


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

With the embeddings of all the training data, we can construct a function that takes in an utterance and outputs a prompt with examples with highest cosine similarity with the utterance.

In [20]:
def get_nearest_neighbor_prompt(utterance: str,
                                training_examples: List[Example],
                                embedding_model: str,
                                corpus_embeddings: torch.Tensor,
                                num_samples: int = 10) -> str:
    """YOUR CODE HERE"""
    utt_embeddings = get_corpus(embedding_model, utterance)
    cos_similarity = torch.cosine_similarity(utt_embeddings, corpus_embeddings, dim=1)
    _, top_k_ids = torch.topk(cos_similarity, num_samples)
    nearest_examples = [training_examples[id] for id in top_k_ids]
    prompt = """Given these examples:
"""
    for example in nearest_examples:
        prompt += "Query: " + example.query + "\nRepresentation: " + example.program + "\n"
    prompt += """Output a formal representation of the following query.
Query: """+ utterance + """
Representation:
"""
    return prompt 

Evaluate the similarity based on prompt on the validation data and save your predictions as `similarity_predictions.txt` with one predictions per line of the validation set. With similarity based example selection, we should get at least 20\% exact match on the validation set. Note that there could be some duplicates in the training data because we are working with a version of the data where the location names are normalized to be variables like `m0`.

In [21]:
similarity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=get_nearest_neighbor_prompt,
                                         training_examples=training_examples,
                                         embedding_model='all-MiniLM-L6-v2',
                                         corpus_embeddings=corpus_embeddings)

# Save the predictions as `similarity_predictions.txt`
with open('similarity_predictions.txt', 'w') as f:
    for prediction in similarity_predictions:
        f.write(prediction)
        f.write('\n')

exact_match = evaluate(similarity_predictions, dev_examples)
print(f"Exact match for nearest neighbor prompt: {exact_match}")

  0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:02<01:27,  2.26s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  5%|▌         | 2/40 [00:06<02:01,  3.19s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 98%|█████████▊| 39/40 [01:51<00:04,  4.11s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 40/40 [01:54<00:00,  2.85s/it]

Exact match for nearest neighbor prompt: 0.7


Let's try to improve on nearest neighbor based example search. This part will be more open ended. We will now implement a different example selection method that improves over uniform random selection. You may implement an algorithm `Diverse Demonstrations Improve In-context Compositional Generalization` (https://arxiv.org/abs/2212.06800) or come up with your own example selection method. In the report, describe the algorithm that you implemented and intuitiion of why it may it could be effective.

In [48]:
def construct_diversity_prompt(utterance: str,
                               training_examples: List[Example],
                               embedding_model: str,
                               corpus_embeddings: torch.Tensor,
                               num_samples: int = 10) -> str:
    """YOUR CODE HERE"""
    utt_embeddings = get_corpus(embedding_model, utterance)
    selected_ids = []
    while len(selected_ids) < num_samples:
        cos_similarity = torch.cosine_similarity(utt_embeddings, corpus_embeddings, dim=1)
        _, top_k_ids = torch.topk(cos_similarity, num_samples)
        for id in top_k_ids:
            if id not in selected_ids:
                selected_ids.append(id)
                corpus_embeddings -= corpus_embeddings[id].clone()
                break
    prompt = """Given these examples:
"""
    selected_examples = [training_examples[id] for id in selected_ids]
    for example in selected_examples:
        prompt += "Query: " + example.query + "\nRepresentation: " + example.program + "\n"
    prompt += """Output a formal representation of the following query.
Query: """+ utterance + """
Representation:
"""
    return prompt 

In [49]:
print(construct_diversity_prompt(utterance="what river runs through m0", 
                                 training_examples=training_examples,
                                 embedding_model='all-MiniLM-L6-v2',
                                 corpus_embeddings=corpus_embeddings))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Given these examples:
Query: what rivers are in m0
Representation: answer ( intersection ( river , loc_2 ( m0 ) ) )
Query: which rivers are in m0
Representation: answer ( intersection ( river , loc_2 ( m0 ) ) )
Query: what rivers are in m0
Representation: answer ( intersection ( river , loc_2 ( m0 ) ) )
Query: what rivers are in m0
Representation: answer ( intersection ( river , loc_2 ( m0 ) ) )
Query: what rivers are in m0
Representation: answer ( intersection ( river , loc_2 ( m0 ) ) )
Query: which states does the m0 river run through
Representation: answer ( intersection ( state , traverse_1 ( intersection ( river , m0 ) ) ) )
Query: how many rivers are in m0
Representation: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Query: how many rivers are in m0
Representation: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Query: how many rivers are in m0
Representation: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Query: what are all the rivers in m0
R

In [51]:
diversity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=construct_diversity_prompt,
                                         training_examples=training_examples,
                                         embedding_model='all-MiniLM-L6-v2',
                                         corpus_embeddings=corpus_embeddings)

# Save the predictions as `diversity_predictions.txt`
with open('diversity_predictions.txt', 'w') as f:
    for prediction in diversity_predictions:
        f.write(prediction)
        f.write('\n')

exact_match = evaluate(diversity_predictions, dev_examples)
print(f"Exact match for diversity based prompt: {exact_match}")

  0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:03<02:17,  3.53s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  5%|▌         | 2/40 [00:07<02:30,  3.97s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 98%|█████████▊| 39/40 [02:28<00:04,  4.58s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 40/40 [02:31<00:00,  3.78s/it]

Exact match for diversity based prompt: 0.675


In [59]:
# Show some examples 
utterance = "what river runs through m0"
parse = parse_example(model="gpt-3.5-turbo",
                      utterance=utterance,
                      prompt_method=random_sample_prompt,
                      training_examples=training_examples)
print(parse)

parse = parse_example(model="gpt-3.5-turbo",
                      utterance=utterance,
                      prompt_method=get_nearest_neighbor_prompt,
                      training_examples=training_examples,
                      embedding_model='all-MiniLM-L6-v2',
                      corpus_embeddings=corpus_embeddings)
print(parse)

parse = parse_example(model="gpt-3.5-turbo",
                      utterance=utterance,
                      prompt_method=construct_diversity_prompt,
                      training_examples=training_examples,
                      embedding_model='all-MiniLM-L6-v2',
                      corpus_embeddings=corpus_embeddings)
print(parse)

answer (intersection (river, m0))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

answer ( traverse_1 ( intersection ( river , loc_2 ( m0 ) ) ) )


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

answer ( traverse_2 ( intersection ( river , loc_2 ( m0 ) ) ) )


Get the predictions and submit these as `diversity_predictions.txt`, where each line is a prediction for the development set. With the improved selection, we should get least 35\% exact match on the validation set.

For the report, compare the predictions from the example selection methods using 1) uniform random sampling 2) embedding-based similarity search and 3) coverage based selection. Compare and contrast the errors and submit your analysis as `report.pdf`

* hw5.ipynb (this file; please rename to match)
* random_predictions.txt
* similarity_predictions.txt
* diversity_predictions.txt
* report.pdf